In [ ]:
from prefect import get_client
import matplotlib.pyplot as plt
import os
from tiled.client import from_uri
import numpy as np

In [ ]:
# Initialize the Tiled server
TILED_URI = os.getenv("TILED_URI")
TILED_API_KEY = os.getenv("TILED_API_KEY")

tiled_client = from_uri(TILED_URI, api_key=TILED_API_KEY)
TILED_BASE_URI = tiled_client.uri

In [ ]:
client = get_client()
deployment = await client.read_deployment_by_name(
    "pixel-roi-vertical-sum-tiled/pixel-roi-vertical-sum"
)

In [ ]:
parameters = {
    "input_uri_data": "raw/cali_saxs_agbh_00001/lmbdp03/cali_saxs_agbh_00001",
    "input_uri_mask": "raw/masks/saxs_mask",
    "beamcenter_x": 900,  # x-coordiante of the beam center postion in pixel
    "beamcenter_y": 2945,  # y-coordiante of the beam center postion in pixel
    "sdd": 833.8931,  # sample-detector-distance in mm beamcenter_y: 2945.038
    "pix_size": 55,  # pixel size in microns
    "wl": 1.05,  # wavelength in Angstrom
    "incident_angle": 0.4,  # incident angle
    "output_unit": "pixel",
    "cut_half_width": 10,
    "ymin": 2744,
    "ymax": 3244,
    "output_key": "test"
    # "rotation": 0,  # detector rotation in degrees (Fit2D convention)
    # "tilt": 0,  # detector tilt in degrees (Fit2D convention)
}

In [ ]:
flow_run = await client.create_flow_run_from_deployment(
    deployment.id, parameters=parameters
)
flow_run_id = flow_run.id

In [ ]:
flow_run = await client.read_flow_run(flow_run_id)
flow_run.state

In [ ]:
data = from_uri(TILED_BASE_URI + parameters["input_uri_data"])[:]
mask = from_uri(TILED_BASE_URI + parameters["input_uri_mask"])[:]
from conversions import mask_image

In [ ]:
plt.imshow(mask_image.fn(data, np.rot90(mask)), origin="lower")